In [ ]:
import pandas as pd
import numpy as np
import cv2
import os

In [ ]:
home_dir = os.getenv("HOME")
raw_data = pd.read_csv(home_dir + '/Desktop/DataPacket/data.csv')

In [ ]:
raw_data.head()

In [ ]:
laser_data = raw_data.drop(["steering_angle", "speed"], axis=1)

laser_data = laser_data.values

maximum_laser_range = 12

for i in range(len(laser_data)):
    for j in range(len(laser_data[i])):
        if np.isinf(laser_data[i][j]):
            laser_data[i][j] = 12.0
            
laser_data = laser_data / 12

In [ ]:
speed = raw_data["speed"].values
steering_angle = raw_data["steering_angle"].values

output = np.dstack((steering_angle, speed))[0]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(laser_data, output, test_size=0.2, random_state=22)

del laser_data
del raw_data
del steering_angle

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, Concatenate, Input
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers.core import Activation

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
model = Sequential()

model.add(Dense(180, input_dim=X_train.shape[1], activation ='relu'))
model.add(Dense(180, activation = "relu"))
model.add(Dense(360, activation = "relu"))
model.add(Dense(360, activation = "relu"))
model.add(Dense(180, activation = "relu"))
model.add(Dense(90, activation = "relu"))
model.add(Dense(45, activation = "relu"))
model.add(Dense(10, activation = "relu"))
model.add(Dense(2, activation = "softmax"))

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [ ]:
hs = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=64, epochs=15)

In [ ]:
import plotly
from plotly.offline import iplot
plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go

def trainingResultsGraph(model, modelCode):
    trace0 = go.Scatter(
        x = model.epoch,
        y = model.history['loss'],
        mode = 'lines',
        name = 'loss',
        line=dict(color='aquamarine')
    )

    trace1 = go.Scatter(
        x = model.epoch,
        y = model.history['val_loss'],
        mode = 'lines',
        name = 'val_loss',
        line=dict(color='darkred', dash='dash')
    )

    trace2 = go.Scatter(
        x = model.epoch,
        y = model.history['acc'],
        mode = 'lines',
        name = 'acc',
        line=dict(color='violet')
    )

    trace3 = go.Scatter(
        x = model.epoch,
        y = model.history['val_acc'],
        mode = 'lines',
        name = 'val_acc',
        line=dict(color='aqua', dash='dash')
    )

    updatemenus = list([
        dict(type="buttons",
             active=-1,
             buttons=list([
                dict(label = 'Acc Graph',
                     method = 'update',
                     args = [{'visible': [False, False, True, True]},
                             {'title': 'Trained Model'+modelCode+' training and validation accuracy'}]),
                dict(label = 'Loss Graph',
                     method = 'update',
                     args = [{'visible': [True, True, False, False]},
                             {'title': 'Trained Model'+modelCode+' training and validation loss'}]),
                dict(label = 'Both',
                     method = 'update',
                     args = [{'visible': [True, True, True, True]},
                             {'title': 'Trained Model'+modelCode+' training and validation values'}])
            ]),
        )
    ])

    data = [trace0, trace1, trace2, trace3]
    layout = dict(title='Trained Model'+modelCode+' training and validation values',
                  xaxis = dict(title = 'Epochs'),
                  updatemenus=updatemenus)

    fig = dict(data=data, layout=layout)

    iplot(fig, filename='lossGraph')
    
trainingResultsGraph(hs, "1")

In [ ]:
try:
    import rospkg
    import rospy

    rospack = rospkg.RosPack()
    package_path = rospack.get_path('angelshark_deeplearning')
    
except ImportError:
    package_path = home_dir + "/Desktop"
    
    import os
    if not os.path.exists(package_path + "/model"):
        os.makedirs(package_path + "/model")
    
    print("Model saved to Desktop/model dir!!!")
    print("Please move them to deeplearning package model directory to angelshark_deeplearning/model")

In [ ]:
import json
import matplotlib.pylab as plt
import random
from keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

# Save model weights and json.
model.save_weights(package_path + '/model/model.h5')
model_json = model.to_json()
with open(package_path + '/model/model.json', 'w') as outfile:
    json.dump(model_json, outfile)

# Train and validation loss chart
print(hs.history.keys())

plt.plot(hs.history['loss'])
plt.plot(hs.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()
